In [1]:
# Necessary imports
import nltk
import numpy as np
import pandas as pd
import tensorflow as ts
import matplotlib.pyplot as plt

In [5]:
df_pos = pd.read_fwf('positive_clean.txt')
df_neg = pd.read_fwf('negative_clean.txt')

KeyboardInterrupt: 

In [ ]:
df_pos.head()

In [ ]:
df_neg.head()